In [ ]:
# 하둡으로 저장된 파일 불러오기
f2011 = spark.read.format("csv").load("/user/ubuntu/homework/flights/csv/2011-summary.csv")   #옵션을 안걸었음
f2011 = spark.read.option("header", "true").csv("homework/flights/csv/2011-summary.csv") 

In [ ]:
# 하둡으로 불어온 f2011로 테이블 만들기
f2011.createOrReplaceTempView("flights2011")

In [ ]:

# dataframe 에서 불러오기
f2011.groupBy("DEST_COUNTRY_NAME").count().show()

In [ ]:
#테이블에서 불러오기
spark.sql("SELECT DEST_COUNTRY_NAME, COUNT(1) FROM flights2011 GROUP BY DEST_COUNTRY_NAME").show()

In [ ]:
#테이블에서 불러오기
spark.sql("SELECT MAX(count) FROM flights2011").show()

In [ ]:
# dataframe 에서 불러오기
from pyspark.sql.functions import max
f2011.select(max("count")).show()

In [ ]:
#테이블에서 불러오기
spark.sql("SELECT  DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME FROM flightS2011  LIMIT 5" ).show()

In [ ]:
# dataframe 에서 불러오기
f2011.select("DEST_COUNTRY_NAME", "ORIGIN_COUNTRY_NAME").show(5)

In [ ]:
# 테이블에서 불러오기
spark.sql("select DEST_COUNTRY_NAME from flights2011 limit 5").show()

In [ ]:
# dataframe 에서 불러오기
from pyspark.sql.functions import col
f2011.select(col("DEST_COUNTRY_NAME")).show(5)


In [ ]:
# 테이블에서 불러오기
spark.sql("SELECT DEST_COUNTRY_NAME as destination  FROM flights2011").show(5)

In [ ]:
# 데이터 프레임에서 불러오기
from pyspark.sql.functions import expr
f2011.select(expr("DEST_COUNTRY_NAME AS destination")).show(5)
f2011.select(col("DEST_COUNTRY_NAME").alias("destination")).s+how(5)
f2011.select(expr("DEST_COUNTRY_NAME").alias("destination")).show(5)


In [ ]:
# selectExpr = select + expr
f2011.selectExpr("DEST_COUNTRY_NAME as destination").show(5)

In [ ]:
# 조건걸어보기!!

In [ ]:
# 테이블에서 불러오기
spark.sql("SELECT *, (DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as DOMESTIC_FLIGHT FROM flights2011").show()

In [ ]:
# 데이터프레임에서 불러오기
f2011.selectExpr("*", "(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as DOMESTIC_FLIGHT").show(100)

In [ ]:
#테이블에서 불러오기
spark.sql("SELECT AVG(count), COUNT(DISTINCT(DEST_COUNTRY_NAME)) FROM flights2011").show()

In [ ]:
# 데이터프레임에서 불러오기
f2011.selectExpr("avg(count)", "count(distinct(DEST_COUNTRY_NAME))").show()


In [ ]:
# lit = literal (값)
# 모든 행에 1이라는 값을 넣어주기!!

In [ ]:

#테이블에서 불러오기
spark.sql("SELECT *, 1 as one FROM flights2011").show(100)

In [ ]:
# 데이터 프레임에서 불러오기
from pyspark.sql.functions import lit
f2011.select(expr("*"), lit(1).alias("one")).show(100)

In [ ]:
#응용한다면 이런식도 됨
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit
import datetime
f2011.select(expr("*"),lit(datetime.datetime.now().strftime('%Y%m%d'))).show(100)
f2011.select(expr("*"),lit(datetime.datetime.now().strftime('%Y%m%d')).alias('load_dt')).show(100)

In [ ]:

# withColumn("컬럼명", "표현식")
# 컬럼을 추가할껀데 어떤 표현식을 가지는 컬럼명을 추가한다.

In [ ]:
#테이블에서 불러오기
spark.sql("SELECT *, (DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) AS DOMESTIC_FLIGHT FROM flights2011").show()

In [ ]:

# 데이터 프레임에서 불러오기
f2011.withColumn("DOMESTIC_FLIGHT", expr("DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME")).show()

In [ ]:
# withColumnRenamed
# 원본 컬럼이름을 바꿀 컬럼이름으로 바꿔주겠다.즉 이름만 바꾼다.

In [ ]:
#  테이블에서 불러오기
spark.sql("select DEST_COUNTRY_NAME as DESTINATION, ORIGIN_COUNTRY_NAME,count  from flights2011").show(5)


In [ ]:
# 데이터 프레임에서 불러오기
f2011.withColumnRenamed("DEST_COUNTRY_NAME", "DESTINATION").show(5)

In [ ]:
withColumn, withColumnRenamed  둘다 원본반영안됨!.따라서 새로운 변수를 만들어줘야 한다.

In [ ]:
# 컬럼 확인용
f2011.columns

In [ ]:
# 테이블에서 가져오기
spark.sql("SELECT * FROM flights2011 WHERE count < 2").show(5)

In [ ]:
# 데이터 프레임에서 가져오기
f2011.filter(col("count") < 2).show(5)
f2011.where("count < 2").show(5)

In [ ]:

#테이블에서 불러오기
spark.sql("SELECT * FROM flights2011 WHERE count < 2 AND ORIGIN_COUNTRY_NAME != 'Croatia'").show(5)

In [ ]:
# 데이터 프레임에서 불러오기
f2011.where("count < 2").where(col("ORIGIN_COUNTRY_NAME") != "Croatia"). show(5)


In [ ]:
#  where 여러개 조건 연결 -> 스파크에서는 조건이 여러개 연결되면 하나하나 순서대로 실행된다가 아니라 동시에 모든 조건 실행(병렬실행)-->즉 동시에 모든 조건 실행한다. => 특정 상황에서 원하지 않는 결과가 도출 될 수 있다!!!

In [ ]:
# 다른결과가 나온다.위에꺼와 같게하려면 어찌해야될까..
f2011.where(("count < 2") and (col("ORIGIN_COUNTRY_NAME") != "Croatia")). show(5)

In [ ]:
#테이블에서 가져오기
spark.sql("""SELECT COUNT(DISTINCT(ORIGIN_COUNTRY_NAME, DEST_COUNTRY_NAME)) AS count FROM flights2011""").show(5)
파이썬에서는 """ """ 는 줄바꿈하더라도 연속적인코드임을 알수있게끔할떄.이꺼쓰기 싫으면 걍 아래같이 쭉 한줄로 써야할꺼다. 
spark.sql("SELECT COUNT(DISTINCT(ORIGIN_COUNTRY_NAME, DEST_COUNTRY_NAME)) AS count FROM flights2011").show()

In [ ]:
# 데이터프레임에서 가져오기
f2011.select("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME").distinct().count()
# count해서 show()는 에러남..

In [ ]:

# row함수
from pyspark.sql import Row
# 새로운 스키마 생성
schema = f2011.schema
#새로운 테이블생성
new_rows = [
	Row("Korea", "Korea", 5),
	Row("Korea", "Australia", 1)
]

# 테이블가져오기
paralle_rows = sc.parallelize(new_rows)
# 데이터 프레임만들기
new_df = spark.createDataFrame(paralle_rows, schema)

f2011.union(new_df).show(10000)